# 1. Import các thư viện cần thiết

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, json, csv, re, os

# 2. Thu thập dữ liệu

[Kickstarter](https://www.kickstarter.com) là trang web cho phép người dùng gọi vốn từ cộng đồng để phát triển một sản phẩm nào đó thuộc lĩnh vực như là: nghệ thuật, truyện tranh, chế tác, phim ảnh, thực phẩm, báo chí, âm nhạc, game, thiết kế, ...

Trong đồ án, nhóm chúng em sẽ thu thập môt số dữ liệu về các dự án hiện có trên nền tảng kickstarter. Theo như file [robots.txt](https://www.kickstarter.com/robots.txt) của kickstarter thì trang không cho phép thu thập thông tin về người hỗ trợ, số tiền đã được góp vốn, widget trong từng trang dự án và trang hồ sơ người dùng, còn lại thì không bị cấm. Nhóm thực hiện thu thập trên trang discover nên không vi phạm yêu cầu của kickstarter.

Ở đây nhóm sử dụng thư viện Selenium vì cần phải thực hiện một số thao tác click chuột và cuộn trang thì kickstarter mới tải thêm dữ liệu.

- Sử dụng thư viện Selenium truy cập trang [discover](https://www.kickstarter.com/discover?ref=nav) của kickstarter và thu thập thông tin của tất cả project hiện có.
- Dữ liệu của các dự án được thu thập bao gồm: id, tên, lời giới thiệu, số tiền mục tiêu, số tiền đã kêu gọi, quốc gia, hạn chót, ngày tạo, ngày khởi động, số người tham gia hỗ trợ, tỉ giá ngoại tê, thể loại, thể loại chính, phần trăm mục tiêu đạt được và kết quả gọi vốn của dự án.
- Dữ liệu thu thập được lưu vào file projects_raw.tsv

In [ ]:
# tạo file tsv lưu dữ liệu
projects_data = open('projects_raw.tsv', 'w', newline='', encoding='utf-8')

In [ ]:
# tạo danh sách các cột
columns = ['id','name','blurb','goal','pledged','country','deadline','created_at','launched_at',
           'backers_count','fx_rate','category','main_category','percent_funded','state']

# tạo danh sách các cột có thể thu thập trực tiếp
cols_collect_data = columns.copy()
cols_collect_data.remove('category')
cols_collect_data.remove('main_category')

In [ ]:
# tạo đối tượng ghi file và ghi vào tên cột
writer = csv.DictWriter(projects_data, fieldnames=columns,delimiter='\t')
writer.writeheader()

In [ ]:
# tạo đối tượng webdriver và truy cập trang kickstarter.com mục discover
# browser = webdriver.Chrome(executable_path = './browser_drivers/chromedriver.exe')
browser = webdriver.Edge(executable_path = './browser_drivers/msedgedriver.exe')
browser.get('https://www.kickstarter.com/discover?ref=nav')

In [ ]:
# parse trang vừa tải
html_tree = BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
# tìm đường dẫn tới những thể loại, lĩnh vực
categories_ls = []
cat_filter = html_tree.find('select', {'name':'category_id'})

In [ ]:
# tìm tag option và có thuộc tính data-urls
# kiểm tra tag option có thuộc tính data-parent-id hay không
# nếu có thì thêm vào danh sách đường dẫn
option = cat_filter.find('option', {'data-urls':True})
while True:
    main_cat = False

    try:
        option['data-parent-id']
    except:
        main_cat = True

    if not main_cat:
        categories_ls.append(re.search('"(http:.+)"', option['data-urls']).group(1))

    option = option.next_sibling
    if option == None:
        break

In [ ]:
# tạo từ điển lưu dữ liệu thu thập được
data_collect = dict.fromkeys(cols_collect_data)

# truy cập trang của từng thể loại chính và thu thập dữ liệu
for category_link in categories_ls:
    time.sleep(5) # đợi 1 khoảng thời gian trước khi tải trang mới

    # tải trang và tìm nút 'load more'
    browser.get(category_link)
    browser.find_element_by_class_name('load_more').click()

    # cuộn tới cuối trang để tải thêm project
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            time.sleep(5)

        last_height = new_height

    # sử dụng BeautifulSoup để parse trang web
    html_tree = BeautifulSoup(browser.page_source, 'html.parser')

    # tìm tất cả các tag chứa thông tin về các project
    project_cards = html_tree.find_all('div', {'class': 'js-react-proj-card'})

    # với mỗi project tìm thuộc tính data-project và thu thập dữ liệu
    for project in project_cards:
        data = json.loads(project['data-project'])

        # thu thập dữ liệu có thể thu thập trực tiếp
        for attr in cols_collect_data:
            data_collect[attr] = data[attr]

        data_collect['category'] = data['category']['name']
        data_collect['main_category'] = data['category']['parent_name']

        # ghi dữ liệu thu thập dược vào file
        writer.writerow(data_collect)

    # lưu dữ liệu thu thập được vào ổ cứng
    projects_data.flush()
    os.fsync(projects_data.fileno())

    project_cards.clear()

In [ ]:
# đóng trình duyệt và đóng file
browser.quit()
projects_data.close()

# 3. Khám phá dữ liệu